In [48]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cv2
%matplotlib inline
from google.colab import drive
from sklearn.preprocessing import  MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
import pandas as pd
from tensorflow.keras.models import Model
# drive.mount("gdrive")
# print(tf.__version__)

2.2.0-rc3
Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).
2.2.0-rc3


In [0]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist().load_data()

train_image = np.reshape(train_images,(-1,28,28,1))/255.
train_label = tf.keras.utils.to_categorical(train_labels)
data = tf.data.Dataset.from_tensor_slices((train_image,train_label)).shuffle(1024).batch(600)
data_aug = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)])

def cnn():
  inp = Input(shape=(28,28,1))
  aug = data_aug(inp)
  cn0 = Conv2D(64,(3,3),activation="relu")(aug)
  cn0 = MaxPooling2D((2,2))(cn0)
  bn0 = BatchNormalization(axis=-1)(cn0)
  cn1 = Conv2D(128,(3,3),activation="relu")(bn0)
  cn1 = MaxPooling2D((2,2))(cn1)
  bn1 = BatchNormalization(axis=-1)(cn1)

  den = Flatten()(bn1)
  den = Dense(10)(den)
  den = Activation("softmax")(den)

  build = Model(inp,den)
  return build

In [0]:
mod = cnn()
optim = tf.keras.optimizers.Nadam(1e-4)
@tf.function
def train_batch(x,y):
  with tf.GradientTape() as t:
    funcl = mod(x)
    lossl = tf.keras.losses.categorical_crossentropy(y,funcl)
    grad = t.gradient(lossl,mod.trainable_variables)
    optim.apply_gradients(zip(grad,mod.trainable_variables))
  return tf.reduce_mean(lossl)

In [0]:
def start_train(epochs):
  for _ in range(epochs):
    for s,(xin,yout) in enumerate(data):
      out = train_batch(xin,yout)
      print("step %s | loss : %.2f"%(s,out))

# start_train(3)